In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import emoji

import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emojify/emojify_data.csv
/kaggle/input/emojify/train_emoji.csv
/kaggle/input/emojify/test_emoji.csv


## 1. load GloVe Embdeing
---

In [2]:
# download stanford GloVe
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2024-07-30 11:54:11--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-07-30 11:54:12--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-07-30 11:54:12--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: 'glove.6B.zip'

glov

In [3]:
def load_glove_model(glove_file):
    print("Loading GloVe Model...")
    model = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array([float(val) for val in split_line[1:]])
            model[word] = embedding
    print("Done.", len(model), " words loaded!")
    return model

glove_model = load_glove_model("glove.6B.300d.txt")  


Loading GloVe Model...
Done. 400000  words loaded!


In [24]:
print(glove_model["<unc>"])

KeyError: '<unc>'

## 2. Data
---

### 2.1 load data

In [13]:
emoji_dictionary = {
    '0' : ':red_heart:',
    '1' : ':baseball:',
    '2' : ':beaming_face_with_smiling_eyes:',
    '3' : ':downcast_face_with_sweat:',
    '4' : ':fork_and_knife:'
    
}
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [14]:
data = pd.read_csv("/kaggle/input/emojify/emojify_data.csv",usecols = [0, 1],  header=None, names=["text", "type"])

data.head

<bound method NDFrame.head of                             text  type
0    French macaroon is so tasty     4
1               work is horrible     3
2                     I am upset     3
3                 throw the ball     1
4                      Good joke     2
..                           ...   ...
178         lets brunch some day     4
179                dance with me     2
180               she is a bully     3
181           she plays baseball     1
182  I like it when people smile     2

[183 rows x 2 columns]>

In [21]:
maxx = 0
for  x in data["text"] : 
    maxx = len(x) if (len(x) > maxx ) else maxx
        
print (maxx)

52


In [ ]:
def embed_text